# PromoterMotifFinder

This program identifies the most common DNA sequence motifs within the promoter regions of a given list of genes. It extracts upstream promoter sequences (typically a fixed length upstream of the transcription start site) from a reference genome and search motifs (with JASPAR database)

Each motif is then counted based on the number of occurrences across all input gene regions. Finally, the motifs are ranked and listed according to their frequency, reflecting how often each motif appears within the gene set.
*   The results are based solely on motif occurrence counts.
*   This analysis does not include any statistical significance testing.


  

In [ ]:
#@title Set parameters and `Runtime` -> `Run all`
from google.colab import files
import os
import re
import hashlib
import random
import glob

from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

SPECIES = "C.elegans" #@param {type:"string"} ["C.elegans", "Mouse", "Killifish"]
#@markdown - Promoter size to search.
PROMOTER_LENGTH = 500 #@param {type:"raw"}
#@markdown - motif match threshold.
PWM_SCORE_THRESHOLD = 0.9 # @param {type:"slider", min:0, max:1, step:0.05}
#@markdown - List of interested genes (WBid/ENSEMBLGENE). must be this format "["Gene1", "Gene2"...]"
GENES_OF_INTEREST = [    "WBGene00022842", "WBGene00014148", "WBGene00014083", "WBGene00014036", "WBGene00014031", "WBGene00014014", "WBGene00013937", "WBGene00022564", "WBGene00022547", "WBGene00010178", "WBGene00022394", "WBGene00013508", "WBGene00013380", "WBGene00022016", "WBGene00013268", "WBGene00013219", "WBGene00013127", "WBGene00013116", "WBGene00012997", "WBGene00012979", "WBGene00012885", "WBGene00012747", "WBGene00012468", "WBGene00021191", "WBGene00012252", "WBGene00011625", "WBGene00006922", "WBGene00020507", "WBGene00015449", "WBGene00021826", "WBGene00010094", "WBGene00016971", "WBGene00013538", "WBGene00006386", "WBGene00011896", "WBGene00011791", "WBGene00020351", "WBGene00020349", "WBGene00020313", "WBGene00011562", "WBGene00020207", "WBGene00006215", "WBGene00006211", "WBGene00013858", "WBGene00005949", "WBGene00005932", "WBGene00005666", "WBGene00005690", "WBGene00005259", "WBGene00005353", "WBGene00016696", "WBGene00022807", "WBGene00005047", "WBGene00004945", "WBGene00017265", "WBGene00011111", "WBGene00017016", "WBGene00004759", "WBGene00017752", "WBGene00004503", "WBGene00004478", "WBGene00004476", "WBGene00004498", "WBGene00004484", "WBGene00004480", "WBGene00004417", "WBGene00004439", "WBGene00009508", "WBGene00004341", "WBGene00007167", "WBGene00003803", "WBGene00004286", "WBGene00004282", "WBGene00020022", "WBGene00019948", "WBGene00011018", "WBGene00019821", "WBGene00004217", "WBGene00004231", "WBGene00016023", "WBGene00004043", "WBGene00008431", "WBGene00008902", "WBGene00003923", "WBGene00016753", "WBGene00020915", "WBGene00003654", "WBGene00003718", "WBGene00003568", "WBGene00007029", "WBGene00003476", "WBGene00003418", "WBGene00007023", "WBGene00007019", "WBGene00020287", "WBGene00003076", "WBGene00002997", "WBGene00003019", "WBGene00019069", "WBGene00002637", "WBGene00010787", "WBGene00019624", "WBGene00010670", "WBGene00010636", "WBGene00019379", "WBGene00002180", "WBGene00009089", "WBGene00002138", "WBGene00002117", "WBGene00013122", "WBGene00002042", "WBGene00002035", "WBGene00001867", "WBGene00010353", "WBGene00001777", "WBGene00001683", "WBGene00012461", "WBGene00009246", "WBGene00001518", "WBGene00001394", "WBGene00022798", "WBGene00019055", "WBGene00010243", "WBGene00018912", "WBGene00018896", "WBGene00010083", "WBGene00010061", "WBGene00010009", "WBGene00018361", "WBGene00018316", "WBGene00009588", "WBGene00009449", "WBGene00009305", "WBGene00017926", "WBGene00009169", "WBGene00017816", "WBGene00017771", "WBGene00009031", "WBGene00017489", "WBGene00017304", "WBGene00017303", "WBGene00008590", "WBGene00008545", "WBGene00001232", "WBGene00001215", "WBGene00004140", "WBGene00011834", "WBGene00010484", "WBGene00007488", "WBGene00001007", "WBGene00014075", "WBGene00007536", "WBGene00000898", "WBGene00008383", "WBGene00008370", "WBGene00008499", "WBGene00019473", "WBGene00011698", "WBGene00000883", "WBGene00000880", "WBGene00000840", "WBGene00010303", "WBGene00006441", "WBGene00019738", "WBGene00010631", "WBGene00009254", "WBGene00008286", "WBGene00016731", "WBGene00016683", "WBGene00016516", "WBGene00016455", "WBGene00007904", "WBGene00015817", "WBGene00015500", "WBGene00007347", "WBGene00007340", "WBGene00015461", "WBGene00007330", "WBGene00015450", "WBGene00007299", "WBGene00000249", "WBGene00000241", "WBGene00007192", "WBGene00015192", "WBGene00015064", "WBGene00010497", "WBGene00013799", "WBGene00016793", "WBGene00000146", "WBGene00015335", "WBGene00000446", "WBGene00022762", "WBGene00013100", "WBGene00013270", "WBGene00012994", "WBGene00020630", "WBGene00009743", "WBGene00005006", "WBGene00007042", "WBGene00003865", "WBGene00003669", "WBGene00003668", "WBGene00003644", "WBGene00006471", "WBGene00017510", "WBGene00015901", "WBGene00003210", "WBGene00002245", "WBGene00013970", "WBGene00019691", "WBGene00009827", "WBGene00021446", "WBGene00001966", "WBGene00001707", "WBGene00010264", "WBGene00010086", "WBGene00000895", "WBGene00000584", "WBGene00013425", "WBGene00007417", "WBGene00020485", "WBGene00012584", "WBGene00007094", "WBGene00015477", "WBGene00018794", "WBGene00013383", "WBGene00000096", "WBGene00014917", "WBGene00010390", "WBGene00008723", "WBGene00008657", "WBGene00019635", "WBGene00019252", "WBGene00009273", "WBGene00013159", "WBGene00021795", "WBGene00012964", "WBGene00013726", "WBGene00006958", "WBGene00006606", "WBGene00044063", "WBGene00013680", "WBGene00004430", "WBGene00010992", "WBGene00003904", "WBGene00011656", "WBGene00018143", "WBGene00020796", "WBGene00003034", "WBGene00019680", "WBGene00019952", "WBGene00010159", "WBGene00010135", "WBGene00018431", "WBGene00009118", "WBGene00016561", "WBGene00000886", "WBGene00010333", "WBGene00020246", "WBGene00016590", "WBGene00016508", "WBGene00016087", "WBGene00016046", "WBGene00015814", "WBGene00000268", "WBGene00000264", "WBGene00015134", "WBGene00015097", "WBGene00011113", "WBGene00003684", "WBGene00003604", "WBGene00003720", "WBGene00003712",] #@param {type:"raw"}

jaspar_taxid = {"C.elegans": 6239,
                "Killifish": 8078,
                "Mouse": 10090}

jaspar_taxgrp = {"C.elegans": "Nematodes",
                "Killifish": "Vertebrates",
                "Mouse": "Vertebrates",

}
genome_wget_addr = {"C.elegans": "https://ftp.ensembl.org/pub/release-114/fasta/caenorhabditis_elegans/dna/Caenorhabditis_elegans.WBcel235.dna_sm.toplevel.fa.gz",
                    "Killifish": "https://ftp.ensembl.org/pub/release-114/fasta/nothobranchius_furzeri/dna/Nothobranchius_furzeri.Nfu_20140520.dna_rm.toplevel.fa.gz",
                    "Mouse": "https://ftp.ensembl.org/pub/release-114/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna_rm.toplevel.fa.gz",
                    }
gtf_wget_addr = {"C.elegans": "https://ftp.ensembl.org/pub/release-114/gtf/caenorhabditis_elegans/Caenorhabditis_elegans.WBcel235.114.gtf.gz",
                "Killifish": "https://ftp.ensembl.org/pub/release-114/gtf/nothobranchius_furzeri/Nothobranchius_furzeri.Nfu_20140520.114.gtf.gz",
                 "Mouse": "https://ftp.ensembl.org/pub/release-114/gtf/mus_musculus/Mus_musculus.GRCm39.114.gtf.gz",
                 }

target_taxid = jaspar_taxgrp[SPECIES]
GENOME_WGET = genome_wget_addr[SPECIES]
GTF_WGET = gtf_wget_addr[SPECIES]

GeneIDheader = {"C.elegans": "WB",
                "Killifish": "ENSNFUG",
                "Mouse": "ENSMUSG"}

for x in GENES_OF_INTEREST:
    assert GeneIDheader[SPECIES] in x, "Check your gene list again"

# Run

In [ ]:
pip install biopython

In [ ]:
import sys
import requests
from collections import defaultdict, Counter
from Bio import SeqIO # You'll need Biopython installed (pip install biopython)
from Bio import motifs
from Bio.Seq import Seq
# For statistical testing later (conceptual)
# from scipy.stats import fisher_exact

In [ ]:
class JASPARrest:
    def __init__(self, use_progress_bar=False):
        self.endpoint = 'https://jaspar.elixir.no'
        self.headers = {"content-type": "application/json"}
        self.use_progress_bar = use_progress_bar

    def get_pfm(self, taxid):
        _jaspar_motifs = dict()

        url = f"{self.endpoint}/api/v1/taxon/{taxid}?page_size=999"
        response = requests.get(url, headers=self.headers)
        if response.ok:
            data = response.json()['results']
            matrix_ids = [x['matrix_id'] for x in data]

            pfm_url = f"{self.endpoint}/api/v1/matrix"
            for mid in matrix_ids:
                r = requests.get(f"{pfm_url}/{mid}", headers=self.headers)
                pfm_matrix = r.json()
                if r.ok:
                    current_motif_name = f"{pfm_matrix['matrix_id']},{pfm_matrix['name']}"
                    motif_obj = motifs.Motif(alphabet=["A","C","G","T"], counts=pfm_matrix['pfm'])
                    motif_obj.name = current_motif_name
                    _jaspar_motifs[current_motif_name] = motif_obj

            return _jaspar_motifs
        else:
            raise('Connection to JASPAR, failed!')

    def browse(self, taxid):
        url = f"{self.endpoint}/api/v1/tffm/{taxid}"
        response = requests.get(url, headers=self.headers)
        print(response.json())

#rest_jaspar = JASPARrest()
#jaspar_motifs = rest_jaspar.get_pfm(target_taxid)
#print(jaspar_motifs)

In [ ]:
!wget $GENOME_WGET -O genome.fa.gz
!gunzip -f genome.fa.gz
!wget $GTF_WGET -O gtf.gz
!gunzip -f gtf.gz

In [ ]:
# --- Configuration ---
GENOME_FASTA = "/content/genome.fa"  # Replace with your C. elegans genome FASTA file
REFSEQ_GTF = "/content/gtf" # Replace with your C. elegans GTF/GFF file (e.g., from NCBI/Ensembl)

In [ ]:
# --- 1. Load Genome ---
def load_genome(fasta_file):
    """
    Loads the genome sequence from a FASTA file into a dictionary.
    Keys are chromosome names, values are SeqIO objects.
    """
    print(f"Loading genome from {fasta_file}...")
    genome = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
    print("Genome loaded.")
    return genome

# --- 2. Parse Gene Annotations and Get Promoter Coordinates ---
def get_promoter_coordinates(gtf_file, genes_of_interest, promoter_length):
    """
    Parses a GTF/GFF file to get gene coordinates and calculate promoter regions.
    Returns a dictionary: {gene_id: {'chr': str, 'start': int, 'end': int, 'strand': str}}
    Note: This is a simplified parser. Real GTF/GFF parsing can be complex.
    """
    print(f"Parsing annotations from {gtf_file} and identifying promoter regions...")
    gene_promoters = {}
    with open(gtf_file, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            parts = line.strip().split('\t')
            # Assuming GTF/GFF format: seqname source feature start end score strand frame attributes
            seqname = parts[0]
            feature_type = parts[2]
            start = int(parts[3])
            end = int(parts[4])
            strand = parts[6]
            attributes = parts[8]

            if feature_type == "gene": # Or "CDS" depending on what you define as gene start
                # Extract gene ID from attributes (this part is highly dependent on GTF/GFF format)
                # Example for Ensembl GTF: gene_id "WBGene0000001"; gene_version "1";
                #gene_id_match = [attr.strip().split(' ')[1].strip('"') for attr in attributes.split(';') if "gene_id" in attr]
                gene_id_match = attributes.split(';')[0].replace('gene_id ','').replace('"','')
                if not gene_id_match:
                    continue
                gene_id = gene_id_match

                if genes_of_interest and gene_id not in genes_of_interest:
                    continue

                promoter_start = -1
                promoter_end = -1

                if strand == '+':
                    # Promoter is upstream of the start site
                    promoter_start = start - promoter_length
                    promoter_end = start - 1 # Or just 'start' if you want to include the TSS base
                elif strand == '-':
                    # Promoter is upstream of the end site (for reverse strand)
                    promoter_start = end + 1 # Or just 'end'
                    promoter_end = end + promoter_length

                # Ensure promoter coordinates are not negative
                promoter_start = max(0, promoter_start)

                if promoter_start != -1 and promoter_end != -1:
                    gene_promoters[gene_id] = {
                        'chr': seqname,
                        'start': promoter_start,
                        'end': promoter_end,
                        'strand': strand
                    }
    print(f"Found promoter regions for {len(gene_promoters)} genes.")
    return gene_promoters

# --- 3. Extract Promoter Sequences ---
def extract_promoter_sequences(genome, gene_promoters):
    """
    Extracts the DNA sequence for each promoter region.
    Returns a dictionary: {gene_id: 'sequence'}
    """
    print("Extracting promoter sequences...")
    promoter_sequences = {}
    for gene_id, coords in gene_promoters.items():
        chrom = coords['chr']
        start = coords['start']
        end = coords['end']
        strand = coords['strand']

        if chrom not in genome:
            print(f"Warning: Chromosome '{chrom}' not found in genome for gene {gene_id}. Skipping.")
            continue

        # Biopython Seq objects are 0-indexed, slice is [start:end]
        # Adjusting for 1-based GTF to 0-based Python slicing
        seq_obj = genome[chrom].seq[start:end]

        if strand == '-':
            seq_obj = seq_obj.reverse_complement()
        promoter_sequences[gene_id] = str(seq_obj).upper() # Convert to string and uppercase
    print(f"Extracted sequences for {len(promoter_sequences)} promoters.")
    return promoter_sequences

In [ ]:
# --- Revised Common K-mer to Known Motif Comparison (using PFM/PWM scanning) ---
def compare_common_kmers_with_pfms(common_kmers, jaspar_motifs, promoter_sequences, score_threshold):
    """
    Compares found common k-mers by scanning promoter regions with JASPAR PWMs.
    This is a more robust way to link found k-mers to known motifs.
    It identifies which promoter regions contain matches to known motifs.
    """
    print(f"\n--- Scanning promoter regions for known JASPAR motifs (score threshold: {score_threshold*100:.1f}%) ---")
    motif_hits_in_promoters = defaultdict(lambda: defaultdict(list)) # {motif_name: {gene_id: [(pos, score, seq, strand), ...]}}
    overall_motif_counts = defaultdict(int) # {motif_name: total occurrences}

    for motif_name, motif_obj in jaspar_motifs.items():
        print(f"Scanning for motif: {motif_name} (Length: {len(motif_obj)})")
        for gene_id, promoter_seq in promoter_sequences.items():
            # Ensure promoter sequence is long enough to contain the motif
            if len(promoter_seq) >= len(motif_obj):
                matches = scan_sequence_with_pwm(promoter_seq, motif_obj, score_threshold)
                if matches:
                    motif_hits_in_promoters[motif_name][gene_id].extend(matches)
                    overall_motif_counts[motif_name] += len(matches)

    print("\n--- Summary of Known Motif Hits in Promoters ---")
    if overall_motif_counts:
        for motif_name, count in sorted(overall_motif_counts.items(), key=lambda item: item[1], reverse=True):
            continue#print(f"Motif: {motif_name} (Total hits: {count})")
            # You can uncomment to print individual hits:
            # for gene_id, hits in motif_hits_in_promoters[motif_name].items():
            #     for pos, score, seq, strand in hits:
            #         print(f"  Gene {gene_id}: Match '{seq}' at position {pos} (strand {strand}, score {score:.2f})")
    else:
        print("No known motifs found in promoter regions with the given threshold.")

    # You could also link your *found k-mers* directly to the motifs
    # by checking if your exact k-mers (from common_kmers) achieve a high score
    # against the PWMs, but the primary use of PWMs is to scan a sequence.
    # For now, we'll just report the found motif occurrences.
    return motif_hits_in_promoters


# --- scan_sequence_with_pwm (The latest corrected version with .min_score()/.max_score())
def scan_sequence_with_pwm(sequence, motif_obj, threshold_percent):
    """
    Scans a DNA sequence for matches to a motif using its PWM/PSSM.
    Returns a list of (start_position, score, matching_sequence) tuples
    for matches exceeding the threshold, considering both strands.
    """
    matches = []

    pssm = motif_obj.pssm

    min_score = pssm.min
    max_score = pssm.max

    if (max_score - min_score) > 0:
        score_cutoff = (max_score - min_score) * threshold_percent + min_score
    else:
        # print(f"Warning: Motif '{motif_obj.name}' has min_score == max_score. Skipping scanning.", file=sys.stderr)
        return [] # Return empty if no valid score range

    # Scan forward strand
    for position, score in pssm.search(str(sequence), threshold=score_cutoff):
        if position + len(motif_obj) <= len(sequence):
            matched_seq = sequence[position : position + len(motif_obj)]
            matches.append((position, score, matched_seq, '+'))

    # Scan reverse complement strand
    rev_comp_sequence_obj = Seq(sequence).reverse_complement()
    rev_comp_sequence_str = str(rev_comp_sequence_obj)

    for position, score in pssm.search(rev_comp_sequence_str, threshold=score_cutoff):
        if position + len(motif_obj) <= len(rev_comp_sequence_str):
            matched_seq = rev_comp_sequence_str[position : position + len(motif_obj)]
            matches.append((position, score, matched_seq, '-'))

    return matches

In [ ]:
# --- Main Execution ---
if __name__ == "__main__":
    # --- 1. Load Genome ---
    genome_data = load_genome(GENOME_FASTA)
    if not genome_data:
        print("Error: Genome data could not be loaded. Exiting.")
        sys.exit(1)

    # --- 2. Load JASPAR Motifs ---
    rest_jaspar = JASPARrest()
    jaspar_motifs_obj = rest_jaspar.get_pfm(target_taxid)
    #jaspar_motifs_obj = load_jaspar_pfm(JASPAR_PFM_FILE)
    if not jaspar_motifs_obj:
        print("Error: No JASPAR motifs loaded from the PFM file. Check file path and format. Exiting.")
        sys.exit(1)

    # --- 3. Get Promoter Coordinates for TARGET Genes ---
    print(f"\n--- Processing Target Genes: {len(GENES_OF_INTEREST)} genes specified ---")
    target_promoter_coords = get_promoter_coordinates(REFSEQ_GTF, GENES_OF_INTEREST, PROMOTER_LENGTH)

    if not target_promoter_coords:
        print("Warning: No promoter coordinates found for target genes. Cannot scan for motifs. Exiting.")
        sys.exit(1)

    genes_to_process_target = {gene_id: coords for gene_id, coords in target_promoter_coords.items()
                               if coords['chr'] in genome_data}

    if not genes_to_process_target:
        print("Warning: No target genes to process after filtering by genome presence. Exiting.")
        sys.exit(1)

    # --- 4. Extract Promoter Sequences for TARGET Genes ---
    target_promoter_seqs = extract_promoter_sequences(genome_data, genes_to_process_target)
    if not target_promoter_seqs:
        print("Warning: No promoter sequences could be extracted for target genes. Exiting.")
        sys.exit(1)

    # --- 5. Scan Target Promoter Regions for Known Motifs ---
    print("\n--- Scanning Target Promoter Regions for Known Motifs ---")
    target_motif_hits = defaultdict(lambda: {'count': 0, 'genes': set()}) # {motif_name: {'count': N, 'genes': {gene_ids}}}

    for motif_name, motif_obj in jaspar_motifs_obj.items():
        if not motif_obj or len(motif_obj) == 0:
            continue # Skip invalid motifs

        try:
            _ = motif_obj.pssm # Test PSSM access
        except Exception as e:
            print(f"Warning: Could not generate PSSM for motif '{motif_name}': {e}. Skipping.", file=sys.stderr)
            continue

        # print(f"Scanning for motif: {motif_name} (Length: {len(motif_obj)}) in target promoters...")
        for gene_id, promoter_seq in target_promoter_seqs.items():
            if len(promoter_seq) >= len(motif_obj):
                matches = scan_sequence_with_pwm(promoter_seq, motif_obj, PWM_SCORE_THRESHOLD)
                if matches:
                    target_motif_hits[motif_name]['count'] += len(matches)
                    target_motif_hits[motif_name]['genes'].add(gene_id)
            # else:
            #     print(f"Debug: Promoter for {gene_id} too short for motif {motif_name}. Skipping.")

    print("\n--- Motif Scan Results in Target Promoters ---")
    if target_motif_hits:
        sorted_target_hits = sorted(target_motif_hits.items(), key=lambda item: item[1]['count'], reverse=True)
        for motif_name, data in sorted_target_hits:
            print(f"Motif: {motif_name}, Hits: {data['count']}, Genes with hits: {len(data['genes'])}")
    else:
        print("No motifs found in target promoter regions with the given threshold.")

In [ ]:
import datetime
import pytz
import pandas as pd
from google.colab import files


final = []
for k, v in sorted_target_hits:
    final.append({'motif': k, 'hits': v['count'], 'genes': len(v['genes']), 'gene list': v['genes']})

current_date = datetime.datetime.now().astimezone(pytz.timezone('CET'))
datestamp = f"{str(current_date.year)[-2:]}{current_date.month:02d}{current_date.day:02d}_{current_date.strftime('%X').replace(':','')}"
download_name = f"motif_search_{datestamp}.csv"
pd.DataFrame().from_dict(final).to_csv(download_name)
files.download(f"{download_name}")

# Note
Vibe coding with Gemini & [Youngjun Park](https://github.com/iron-lion) (youngjun.park@age.mpg.de)